<a href="https://colab.research.google.com/github/dr-a-r/multiclass-classification/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import nltk
resources = ['stopwords','punkt','wordnet','averaged_perceptron_tagger']
nltk.download(resources)
from nltk.corpus import stopwords

import seaborn as sns

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Data Collection

In [5]:
data = pd.read_csv('/content/Symptom2Disease.csv')

In [6]:
data = data.drop('Unnamed: 0', axis = 1)
data

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,Psoriasis,"My nails have small dents or pits in them, and..."
...,...,...
1195,diabetes,I'm shaking and trembling all over. I've lost ...
1196,diabetes,"Particularly in the crevices of my skin, I hav..."
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,"I have trouble breathing, especially outside. ..."


### Data Preprocessing

In [7]:
data.isnull().sum()

label    0
text     0
dtype: int64

In [8]:
data.label.value_counts()

label
Psoriasis                          50
Varicose Veins                     50
peptic ulcer disease               50
drug reaction                      50
gastroesophageal reflux disease    50
allergy                            50
urinary tract infection            50
Malaria                            50
Jaundice                           50
Cervical spondylosis               50
Migraine                           50
Hypertension                       50
Bronchial Asthma                   50
Acne                               50
Arthritis                          50
Dimorphic Hemorrhoids              50
Pneumonia                          50
Common Cold                        50
Fungal infection                   50
Dengue                             50
Impetigo                           50
Chicken pox                        50
Typhoid                            50
diabetes                           50
Name: count, dtype: int64

In [9]:
data['text'] = data['text'].str.replace('[^a-zA-Z\s]','',regex=True)
data

,label,text
0,Psoriasis,I have been experiencing a skin rash on my arm...
1,Psoriasis,My skin has been peeling especially on my knee...
2,Psoriasis,I have been experiencing joint pain in my fing...
3,Psoriasis,There is a silver like dusting on my skin espe...
4,Psoriasis,My nails have small dents or pits in them and ...
...,...,...
1195,diabetes,Im shaking and trembling all over Ive lost my ...
1196,diabetes,Particularly in the crevices of my skin I have...
1197,diabetes,I regularly experience these intense urges and...
1198,diabetes,I have trouble breathing especially outside I ...


In [10]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def remove_stopwords(sentence):
    sentence = sentence.lower()
    words = sentence.split()
    fil_words = [word for word in words if word not in stop_words]
    new_sentence = ' '.join(fil_words)
    return new_sentence

data['text'] = data['text'].apply(remove_stopwords)


In [11]:
from nltk.tokenize import word_tokenize

data['tokens'] = data['text'].apply(word_tokenize)

data.head()

,label,text,tokens
0,Psoriasis,experiencing skin rash arms legs torso past we...,"[experiencing, skin, rash, arms, legs, torso, ..."
1,Psoriasis,skin peeling especially knees elbows scalp pee...,"[skin, peeling, especially, knees, elbows, sca..."
2,Psoriasis,experiencing joint pain fingers wrists knees p...,"[experiencing, joint, pain, fingers, wrists, k..."
3,Psoriasis,silver like dusting skin especially lower back...,"[silver, like, dusting, skin, especially, lowe..."
4,Psoriasis,nails small dents pits often feel inflammatory...,"[nails, small, dents, pits, often, feel, infla..."


In [21]:
data.head()

,label,text,tokens,lemma
0,Psoriasis,experiencing skin rash arms legs torso past we...,"[experiencing, skin, rash, arms, legs, torso, ...","[experiencing, skin, rash, arm, leg, torso, pa..."
1,Psoriasis,skin peeling especially knees elbows scalp pee...,"[skin, peeling, especially, knees, elbows, sca...","[skin, peeling, especially, knee, elbow, scalp..."
2,Psoriasis,experiencing joint pain fingers wrists knees p...,"[experiencing, joint, pain, fingers, wrists, k...","[experiencing, joint, pain, finger, wrist, kne..."
3,Psoriasis,silver like dusting skin especially lower back...,"[silver, like, dusting, skin, especially, lowe...","[silver, like, dusting, skin, especially, lowe..."
4,Psoriasis,nails small dents pits often feel inflammatory...,"[nails, small, dents, pits, often, feel, infla...","[nail, small, dent, pit, often, feel, inflamma..."


### Label encoding

In [26]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data['encoded_labels'] = label_encoder.fit_transform(data.label)


In [43]:
encoded_labels = data['encoded_labels']

In [42]:
sorted(data.encoded_labels.unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [29]:
label_encoder.inverse_transform([15,16,13])

array(['Psoriasis', 'Typhoid', 'Migraine'], dtype=object)

### Feature Engineering

In [30]:
# Lemmatization


from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn

lemmatizer = WordNetLemmatizer()

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J']: wn.ADJ
tag_map['V']: wn.VERB
tag_map['R']: wn.ADV



def lemmatize_token_list(token_list):

    # creating tagged tokens
    tagged_tokens = pos_tag(token_list)
    final_words = []

    # lemmatization
    for i in range(len(token_list)):
        token , tag = tagged_tokens[i]
        word_final = lemmatizer.lemmatize(token, tag_map[tag[0]])
        final_words.append(word_final)

    return final_words

# applying text preprocessing to tokens column

data['lemma'] = data['tokens'].apply(lemmatize_token_list)
data.head()

,label,text,tokens,lemma,encoded_labels
0,Psoriasis,experiencing skin rash arms legs torso past we...,"[experiencing, skin, rash, arms, legs, torso, ...","[experiencing, skin, rash, arm, leg, torso, pa...",15
1,Psoriasis,skin peeling especially knees elbows scalp pee...,"[skin, peeling, especially, knees, elbows, sca...","[skin, peeling, especially, knee, elbow, scalp...",15
2,Psoriasis,experiencing joint pain fingers wrists knees p...,"[experiencing, joint, pain, fingers, wrists, k...","[experiencing, joint, pain, finger, wrist, kne...",15
3,Psoriasis,silver like dusting skin especially lower back...,"[silver, like, dusting, skin, especially, lowe...","[silver, like, dusting, skin, especially, lowe...",15
4,Psoriasis,nails small dents pits often feel inflammatory...,"[nails, small, dents, pits, often, feel, infla...","[nail, small, dent, pit, often, feel, inflamma...",15


In [31]:
X = data['lemma']
def join_func(X):
    text = X
    next_text = ' '.join(text)
    return next_text

X = X.apply(join_func)

In [32]:
X

0       experiencing skin rash arm leg torso past week...
1       skin peeling especially knee elbow scalp peeli...
2       experiencing joint pain finger wrist knee pain...
3       silver like dusting skin especially lower back...
4       nail small dent pit often feel inflammatory te...
                              ...                        
1195    im shaking trembling ive lost sense taste smel...
1196    particularly crevice skin skin rash irritation...
1197    regularly experience intense urge want urinate...
1198    trouble breathing especially outside start fee...
1199    constantly sneeze dry cough infection dont see...
Name: lemma, Length: 1200, dtype: object

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Fit and transform the documents to obtain the TF-IDF matrix
X_matrix = vectorizer.fit_transform(X)

In [34]:


# Get the feature names (terms) in the TF-IDF matrix
feature_names = vectorizer.get_feature_names_out()

# Convert the TF-IDF matrix to a dense array for easier inspection (optional)
tfidf_matrix_dense = X_matrix.toarray()



## Model Buildng

In [38]:
encoded_labels.shape

(8,)

In [44]:
model = LogisticRegression(multi_class='ovr')

# Train the model on the train data
model.fit(X_matrix, encoded_labels)


LogisticRegression(multi_class='ovr')

#### Prediction single disease

In [46]:
def predict_label(complaints):
    input = vectorizer.transform([complaints])
    pred = model. predict(input)
    label = label_encoder.inverse_transform(pred)
    return label

In [47]:
predict_label('fever with burning sensation while urinating')

array(['urinary tract infection'], dtype=object)

#### Predicting differential diagnosis

In [48]:
def dif_diagnosis(input):
    input_vec = vectorizer.transform([input])
    predicted_probabilities = model.predict_proba(input_vec)
    top_5_class_indices = np.argsort(predicted_probabilities[0])[::-1][:5]
    top_5_class_labels = label_encoder.inverse_transform(top_5_class_indices)
    return top_5_class_labels



In [49]:
complaints = 'I have stomach pain and fever'
dif_diagnosis(complaints)

array(['Typhoid', 'Jaundice', 'Dengue', 'peptic ulcer disease',
       'Hypertension'], dtype=object)